In [1]:
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb


def load_train():
    train = pd.read_csv('./train.csv')
    labels = train.target.values
    lbl_enc = preprocessing.LabelEncoder()
    labels = lbl_enc.fit_transform(labels)
    train = train.drop('id', axis=1)
    train = train.drop('target', axis=1)
    return np.log(train.values + 1), labels.astype('int32')


def load_test():
    test = pd.read_csv('./test.csv')
    test = test.drop('id', axis=1)
    return np.log(test.values + 1)


def write_submission(preds, output):
    sample = pd.read_csv('./data/sampleSubmission.csv')
    preds = pd.DataFrame(
        preds, index=sample.id.values, columns=sample.columns[1:])
    preds.to_csv(output, index_label='id')

In [2]:
def score(params):
    print ("Training with params : ")
    print (params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round,watchlist,early_stopping_rounds=10)
    predictions = model.predict(dvalid).reshape((X_val.shape[0], 9))
    score = log_loss(y_val, predictions)
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : 300,
             #'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'eta' : 0.1,
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
             #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'subsample' : 0.8,
             #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
             'colsample_bytree' : 0.8,
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 4,
             'silent' : 1
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=20)


In [5]:


X, y = load_train()
split = StratifiedShuffleSplit(y,n_iter=1,test_size=0.2)
for train_idx,val_idx in split:
    pass
X_train = X[train_idx]
y_train = y[train_idx]
X_val = X[val_idx]
y_val = y[val_idx]

In [6]:
optimize(trials)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.952100	eval-mlogloss:1.959280
[1]	train-mlogloss:1.769247	eval-mlogloss:1.783167
[2]	train-mlogloss:1.626536	eval-mlogloss:1.647196
[3]	train-mlogloss:1.508275	eval-mlogloss:1.536381
[4]	train-mlogloss:1.407477	eval-mlogloss:1.441638
[5]	train-mlogloss:1.317170	eval-mlogloss:1.356570
[6]	train-mlogloss:1.237773	eval-mlogloss:1.282531
[7]	train-mlogloss:1.167624	eval-mlogloss:1.216815
[8]	train-mlogloss:1.106196	eval-mlogloss:1.160227
[9]	train-mlogloss:1.052039	eval-mlogloss:1.110350
[10]	train-mlogloss:1.002396	eval-mlogloss:1.064971
[11]	train-mlogloss:0.956095	eval-mlogloss:1.023140
[12]	train-mlogloss:0.914986	eval-mlogloss:0.986056
[13]	train-mlogloss:0.877524	eval-mlogloss:0.952643
[14]	train-mlogloss:0.843298	eval-mlogloss:0.922177
[15]	train-mlogloss:0.810933	eval-mlogloss:0.893125
[16]	train-mlogloss:0.781164	eval-mlogloss:0.866854
[17]	train-mlogloss:0.753748	eval-mlogloss:0.842820
[18]	train-mlog

Training with params : 
{'n_estimators': 300, 'min_child_weight': 9.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.46243553804105203



Stopping. Best iteration:
[226]	train-mlogloss:0.160641	eval-mlogloss:0.462418

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.948938	eval-mlogloss:1.961953
[1]	train-mlogloss:1.764707	eval-mlogloss:1.789276
[2]	train-mlogloss:1.619394	eval-mlogloss:1.656021
[3]	train-mlogloss:1.492023	eval-mlogloss:1.538486
[4]	train-mlogloss:1.382502	eval-mlogloss:1.436942
[5]	train-mlogloss:1.288412	eval-mlogloss:1.350846
[6]	train-mlogloss:1.209399	eval-mlogloss:1.279308
[7]	train-mlogloss:1.138153	eval-mlogloss:1.214138
[8]	train-mlogloss:1.072774	eval-mlogloss:1.156065
[9]	train-mlogloss:1.016928	eval-mlogloss:1.105971
[10]	train-mlogloss:0.964440	eval-mlogloss:1.058973
[11]	train-mlogloss:0.916086	eval-mlogloss:1.016566
[12]	train-mlogloss:0.872770	eval-mlogloss:0.978621
[13]	train-mlogloss:0.832580	eval-mlogloss:0.943973
[14]	train-mlogloss:0.796044	eval-mlogloss:0.912579
[15]	train-mlogloss:0.764322	eval-mlogloss:0.885284
[16]	train-mlogloss:0.734226	eval-mlogl


Training with params : 
{'n_estimators': 300, 'min_child_weight': 2.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 11.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.4669555566647315



[196]	train-mlogloss:0.120943	eval-mlogloss:0.466955
Stopping. Best iteration:
[186]	train-mlogloss:0.127945	eval-mlogloss:0.466763

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:2.035378	eval-mlogloss:2.036381
[1]	train-mlogloss:1.899404	eval-mlogloss:1.903053
[2]	train-mlogloss:1.776832	eval-mlogloss:1.781718
[3]	train-mlogloss:1.677712	eval-mlogloss:1.683616
[4]	train-mlogloss:1.596161	eval-mlogloss:1.603654
[5]	train-mlogloss:1.525711	eval-mlogloss:1.533676
[6]	train-mlogloss:1.466538	eval-mlogloss:1.475141
[7]	train-mlogloss:1.412113	eval-mlogloss:1.421117
[8]	train-mlogloss:1.361714	eval-mlogloss:1.371323
[9]	train-mlogloss:1.319410	eval-mlogloss:1.328997
[10]	train-mlogloss:1.278082	eval-mlogloss:1.288184
[11]	train-mlogloss:1.240113	eval-mlogloss:1.250864
[12]	train-mlogloss:1.204553	eval-mlogloss:1.215612
[13]	train-mlogloss:1.173169	eval-mlogloss:1.184684
[14]	train-mlogloss:1.146207	eval-mlogloss:1.158016
[15]	train-mlogloss:1.118994	eval-mlog

...


Training with params : 
{'n_estimators': 300, 'min_child_weight': 1.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 8.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.4681683890380447



Stopping. Best iteration:
[273]	train-mlogloss:0.151400	eval-mlogloss:0.467907

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.939301	eval-mlogloss:1.952433
[1]	train-mlogloss:1.759804	eval-mlogloss:1.785689
[2]	train-mlogloss:1.611568	eval-mlogloss:1.647091
[3]	train-mlogloss:1.490760	eval-mlogloss:1.535961
[4]	train-mlogloss:1.388701	eval-mlogloss:1.442767
[5]	train-mlogloss:1.295131	eval-mlogloss:1.356920
[6]	train-mlogloss:1.214297	eval-mlogloss:1.283280
[7]	train-mlogloss:1.143010	eval-mlogloss:1.218139
[8]	train-mlogloss:1.079437	eval-mlogloss:1.160092
[9]	train-mlogloss:1.023708	eval-mlogloss:1.110556
[10]	train-mlogloss:0.973913	eval-mlogloss:1.066008
[11]	train-mlogloss:0.926934	eval-mlogloss:1.023872
[12]	train-mlogloss:0.885373	eval-mlogloss:0.986624
[13]	train-mlogloss:0.846234	eval-mlogloss:0.952289
[14]	train-mlogloss:0.810710	eval-mlogloss:0.920815
[15]	train-mlogloss:0.778240	eval-mlogloss:0.892960
[16]	train-mlogloss:0.748663	eval-mlogl


Training with params : 
{'n_estimators': 300, 'min_child_weight': 1.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 10.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.46951816139364877



[196]	train-mlogloss:0.124158	eval-mlogloss:0.469518
Stopping. Best iteration:
[186]	train-mlogloss:0.130750	eval-mlogloss:0.469258

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.977745	eval-mlogloss:1.980716
[1]	train-mlogloss:1.824539	eval-mlogloss:1.830466
[2]	train-mlogloss:1.697374	eval-mlogloss:1.705158
[3]	train-mlogloss:1.592479	eval-mlogloss:1.601397
[4]	train-mlogloss:1.506318	eval-mlogloss:1.516714
[5]	train-mlogloss:1.428406	eval-mlogloss:1.440363
[6]	train-mlogloss:1.360413	eval-mlogloss:1.373271
[7]	train-mlogloss:1.301836	eval-mlogloss:1.316477
[8]	train-mlogloss:1.247952	eval-mlogloss:1.263718
[9]	train-mlogloss:1.197429	eval-mlogloss:1.214660
[10]	train-mlogloss:1.153365	eval-mlogloss:1.171879
[11]	train-mlogloss:1.114627	eval-mlogloss:1.134201
[12]	train-mlogloss:1.077294	eval-mlogloss:1.098040
[13]	train-mlogloss:1.045473	eval-mlogloss:1.066795
[14]	train-mlogloss:1.016394	eval-mlogloss:1.038390
[15]	train-mlogloss:0.988150	eval-mlog


Training with params : 
{'n_estimators': 300, 'min_child_weight': 6.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 5.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.49520023865041957




In [7]:
trials.losses()

[0.46243553804105203,
 0.4669555566647315,
 0.5519025288190725,
 0.4615429077401509,
 0.6070676673454368,
 0.7285325329083832,
 0.49377419816524387,
 0.7286655139916368,
 0.4640815643240792,
 0.6077748260653596,
 0.4827722072009566,
 0.46613880680290204,
 0.4624307711161597,
 0.47920706750642783,
 0.46302652467153343,
 0.4618354245657637,
 0.4625360373794923,
 0.4681683890380447,
 0.46951816139364877,
 0.49520023865041957]

In [8]:
trials.argmin

{'max_depth': 9.0, 'min_child_weight': 6.0}

In [10]:
trials.results

[{'loss': 0.46243553804105203, 'status': 'ok'},
 {'loss': 0.4669555566647315, 'status': 'ok'},
 {'loss': 0.5519025288190725, 'status': 'ok'},
 {'loss': 0.4615429077401509, 'status': 'ok'},
 {'loss': 0.6070676673454368, 'status': 'ok'},
 {'loss': 0.7285325329083832, 'status': 'ok'},
 {'loss': 0.49377419816524387, 'status': 'ok'},
 {'loss': 0.7286655139916368, 'status': 'ok'},
 {'loss': 0.4640815643240792, 'status': 'ok'},
 {'loss': 0.6077748260653596, 'status': 'ok'},
 {'loss': 0.4827722072009566, 'status': 'ok'},
 {'loss': 0.46613880680290204, 'status': 'ok'},
 {'loss': 0.4624307711161597, 'status': 'ok'},
 {'loss': 0.47920706750642783, 'status': 'ok'},
 {'loss': 0.46302652467153343, 'status': 'ok'},
 {'loss': 0.4618354245657637, 'status': 'ok'},
 {'loss': 0.4625360373794923, 'status': 'ok'},
 {'loss': 0.4681683890380447, 'status': 'ok'},
 {'loss': 0.46951816139364877, 'status': 'ok'},
 {'loss': 0.49520023865041957, 'status': 'ok'}]

In [11]:
trials = Trials()
space = {
         'n_estimators' : 500,
         #'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
         'eta' : 0.1,
         'max_depth' : hp.quniform('max_depth', 12, 14, 1),
         'min_child_weight' : hp.quniform('min_child_weight', 5, 9, 1),
         #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
         'subsample' : 0.8,
         #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
         #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
         'colsample_bytree' : 0.8,
         'num_class' : 9,
         'eval_metric': 'mlogloss',
         'objective': 'multi:softprob',
         'nthread' : 4,
         'silent' : 1
         }
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=2)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.949096	eval-mlogloss:1.957793
[1]	train-mlogloss:1.766512	eval-mlogloss:1.782838
[2]	train-mlogloss:1.619591	eval-mlogloss:1.643892
[3]	train-mlogloss:1.497963	eval-mlogloss:1.529184
[4]	train-mlogloss:1.394268	eval-mlogloss:1.431126
[5]	train-mlogloss:1.304623	eval-mlogloss:1.347818
[6]	train-mlogloss:1.228995	eval-mlogloss:1.277294
[7]	train-mlogloss:1.158668	eval-mlogloss:1.212414
[8]	train-mlogloss:1.095939	eval-mlogloss:1.154912
[9]	train-mlogloss:1.038788	eval-mlogloss:1.102736
[10]	train-mlogloss:0.987804	eval-mlogloss:1.056536
[11]	train-mlogloss:0.942273	eval-mlogloss:1.015636
[12]	train-mlogloss:0.901232	eval-mlogloss:0.979158
[13]	train-mlogloss:0.861848	eval-mlogloss:0.944598
[14]	train-mlogloss:0.826618	eval-mlogloss:0.913325
[15]	train-mlogloss:0.794442	eval-mlogloss:0.884991
[16]	train-mlogloss:0.763958	eval-mlogloss:0.858335
[17]	train-mlogloss:0.735380	eval-mlogloss:0.834019
[18]	train-mlog

Training with params : 
{'n_estimators': 500, 'min_child_weight': 7.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.4618006860016755



Stopping. Best iteration:
[212]	train-mlogloss:0.148635	eval-mlogloss:0.461789

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.944109	eval-mlogloss:1.953470
[1]	train-mlogloss:1.761577	eval-mlogloss:1.778665
[2]	train-mlogloss:1.617876	eval-mlogloss:1.641741
[3]	train-mlogloss:1.496784	eval-mlogloss:1.526846
[4]	train-mlogloss:1.398645	eval-mlogloss:1.434152
[5]	train-mlogloss:1.310696	eval-mlogloss:1.351624
[6]	train-mlogloss:1.232240	eval-mlogloss:1.277874
[7]	train-mlogloss:1.162931	eval-mlogloss:1.213810
[8]	train-mlogloss:1.100879	eval-mlogloss:1.156504
[9]	train-mlogloss:1.044221	eval-mlogloss:1.104157
[10]	train-mlogloss:0.994114	eval-mlogloss:1.058550
[11]	train-mlogloss:0.948431	eval-mlogloss:1.017405
[12]	train-mlogloss:0.905923	eval-mlogloss:0.979071
[13]	train-mlogloss:0.868059	eval-mlogloss:0.945336
[14]	train-mlogloss:0.833048	eval-mlogloss:0.914478
[15]	train-mlogloss:0.800750	eval-mlogloss:0.886049
[16]	train-mlogloss:0.770733	eval-mlogl


Training with params : 
{'n_estimators': 500, 'min_child_weight': 8.0, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13.0, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.45847526311852643




[250]	train-mlogloss:0.139074	eval-mlogloss:0.458475
Stopping. Best iteration:
[240]	train-mlogloss:0.144431	eval-mlogloss:0.458430



{'max_depth': 13.0, 'min_child_weight': 8.0}

In [15]:
trials = Trials()
space = {
         'n_estimators' : 500,
         #'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
         'eta' : 0.1,
         'max_depth' : 13,
         'min_child_weight' : 7,
         #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
         'subsample' : 0.8,
         #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
         #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
         'colsample_bytree' : 0.8,
         'num_class' : 9,
         'eval_metric': 'mlogloss',
         'objective': 'multi:softprob',
         'nthread' : 4,
         'silent' : 1
         }
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=1)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.949096	eval-mlogloss:1.957793
[1]	train-mlogloss:1.760465	eval-mlogloss:1.777157
[2]	train-mlogloss:1.614357	eval-mlogloss:1.638632
[3]	train-mlogloss:1.497422	eval-mlogloss:1.529265
[4]	train-mlogloss:1.396328	eval-mlogloss:1.433688
[5]	train-mlogloss:1.306987	eval-mlogloss:1.350147
[6]	train-mlogloss:1.227492	eval-mlogloss:1.277005
[7]	train-mlogloss:1.157049	eval-mlogloss:1.211919
[8]	train-mlogloss:1.094601	eval-mlogloss:1.155054
[9]	train-mlogloss:1.039859	eval-mlogloss:1.104709
[10]	train-mlogloss:0.988806	eval-mlogloss:1.058224
[11]	train-mlogloss:0.941476	eval-mlogloss:1.015877
[12]	train-mlogloss:0.898903	eval-mlogloss:0.977642
[13]	train-mlogloss:0.860823	eval-mlogloss:0.943739
[14]	train-mlogloss:0.825010	eval-mlogloss:0.912324
[15]	train-mlogloss:0.792178	eval-mlogloss:0.883773
[16]	train-mlogloss:0.762294	eval-mlogloss:0.857739
[17]	train-mlogloss:0.733998	eval-mlogloss:0.832625
[18]	train-mlog

Training with params : 
{'n_estimators': 500, 'min_child_weight': 7, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.46221714953102117




[245]	train-mlogloss:0.130232	eval-mlogloss:0.462217
Stopping. Best iteration:
[235]	train-mlogloss:0.135650	eval-mlogloss:0.461859



{}

In [19]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
params =    {
             'eta' : 0.1,
             'max_depth' : 13,
             'min_child_weight' : 7,
             #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'subsample' : 0.8,
             #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
             'colsample_bytree' : 0.8,
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 4,
             'silent' : 0
             }

model = xgb.train(params, dtrain, num_boost_round=500, evals=watchlist,early_stopping_rounds=10)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.949096	eval-mlogloss:1.957793
[1]	train-mlogloss:1.766416	eval-mlogloss:1.782307
[2]	train-mlogloss:1.623759	eval-mlogloss:1.647289
[3]	train-mlogloss:1.499731	eval-mlogloss:1.530014
[4]	train-mlogloss:1.398825	eval-mlogloss:1.435909
[5]	train-mlogloss:1.308401	eval-mlogloss:1.351140
[6]	train-mlogloss:1.228309	eval-mlogloss:1.276633
[7]	train-mlogloss:1.157519	eval-mlogloss:1.211423
[8]	train-mlogloss:1.094222	eval-mlogloss:1.153396
[9]	train-mlogloss:1.037399	eval-mlogloss:1.101632
[10]	train-mlogloss:0.986480	eval-mlogloss:1.055414
[11]	train-mlogloss:0.940409	eval-mlogloss:1.014165
[12]	train-mlogloss:0.898131	eval-mlogloss:0.976107
[13]	train-mlogloss:0.859904	eval-mlogloss:0.942038
[14]	train-mlogloss:0.824629	eval-mlogloss:0.911068
[15]	train-mlogloss:0.792221	eval-mlogloss:0.882811
[16]	train-mlogloss:0.762207	eval-mlogloss:0.856595
[17]	train-mlogloss:0.735451	eval-mlogloss:0.833836
[18]	train-mlog

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'),(dvalid, 'eval')]
params =    {
             'eta' : 0.1,
             'max_depth' : 13,
             'min_child_weight' : 8,
             #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'subsample' : 0.8,
             #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
             'colsample_bytree' : 0.8,
             'num_class' : 9,
             'eval_metric': 'mlogloss',
             'objective': 'multi:softprob',
             'nthread' : 4,
             'silent' : 0
             }

model = xgb.train(params, dtrain, num_boost_round=500, evals=watchlist,early_stopping_rounds=10)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.947744	eval-mlogloss:1.957061
[1]	train-mlogloss:1.760235	eval-mlogloss:1.778323
[2]	train-mlogloss:1.616182	eval-mlogloss:1.642084
[3]	train-mlogloss:1.498197	eval-mlogloss:1.531347
[4]	train-mlogloss:1.393008	eval-mlogloss:1.433758
[5]	train-mlogloss:1.303654	eval-mlogloss:1.351484
[6]	train-mlogloss:1.225800	eval-mlogloss:1.279766
[7]	train-mlogloss:1.155517	eval-mlogloss:1.215129
[8]	train-mlogloss:1.091310	eval-mlogloss:1.156497
[9]	train-mlogloss:1.034759	eval-mlogloss:1.105059
[10]	train-mlogloss:0.984946	eval-mlogloss:1.059834
[11]	train-mlogloss:0.937997	eval-mlogloss:1.017542
[12]	train-mlogloss:0.894676	eval-mlogloss:0.979220
[13]	train-mlogloss:0.855027	eval-mlogloss:0.944165
[14]	train-mlogloss:0.819435	eval-mlogloss:0.912143
[15]	train-mlogloss:0.786459	eval-mlogloss:0.883189
[16]	train-mlogloss:0.756102	eval-mlogloss:0.856715
[17]	train-mlogloss:0.728310	eval-mlogloss:0.832595
[18]	train-mlog

In [21]:
trials = Trials()
space = {
         'n_estimators' : 500,
         #'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
         'eta' : 0.1,
         'max_depth' : 13,
         'min_child_weight' : 8,
         #'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
         #'subsample' : 0.8,
         'subsample' : hp.quniform('subsample', 0.7, 0.9, 0.1),
         #'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
         #'colsample_bytree' : hp.quniform('colsample_bytree', 0.4, 0.9, 0.1),
         'colsample_bytree' : 0.8,
         'num_class' : 9,
         'eval_metric': 'mlogloss',
         'objective': 'multi:softprob',
         'nthread' : 4,
         'silent' : 1
         }
fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=2)

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.950472	eval-mlogloss:1.958747
[1]	train-mlogloss:1.769701	eval-mlogloss:1.785147
[2]	train-mlogloss:1.623827	eval-mlogloss:1.645188
[3]	train-mlogloss:1.501310	eval-mlogloss:1.528508
[4]	train-mlogloss:1.401226	eval-mlogloss:1.434464
[5]	train-mlogloss:1.310637	eval-mlogloss:1.350107
[6]	train-mlogloss:1.234624	eval-mlogloss:1.279073
[7]	train-mlogloss:1.166981	eval-mlogloss:1.216507
[8]	train-mlogloss:1.103648	eval-mlogloss:1.158740
[9]	train-mlogloss:1.047495	eval-mlogloss:1.106862
[10]	train-mlogloss:0.996853	eval-mlogloss:1.061052
[11]	train-mlogloss:0.950048	eval-mlogloss:1.018829
[12]	train-mlogloss:0.907901	eval-mlogloss:0.980806
[13]	train-mlogloss:0.869424	eval-mlogloss:0.946850
[14]	train-mlogloss:0.835610	eval-mlogloss:0.917569
[15]	train-mlogloss:0.803984	eval-mlogloss:0.889944
[16]	train-mlogloss:0.774119	eval-mlogloss:0.863924
[17]	train-mlogloss:0.746647	eval-mlogloss:0.839863
[18]	train-mlog

Training with params : 
{'n_estimators': 500, 'min_child_weight': 8, 'subsample': 0.8, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.460087193437794



[245]	train-mlogloss:0.140026	eval-mlogloss:0.460087
Stopping. Best iteration:
[235]	train-mlogloss:0.145609	eval-mlogloss:0.459723

Will train until eval error hasn't decreased in 10 rounds.
[0]	train-mlogloss:1.943342	eval-mlogloss:1.951485
[1]	train-mlogloss:1.756699	eval-mlogloss:1.773278
[2]	train-mlogloss:1.614305	eval-mlogloss:1.637862
[3]	train-mlogloss:1.491619	eval-mlogloss:1.522185
[4]	train-mlogloss:1.390697	eval-mlogloss:1.428116
[5]	train-mlogloss:1.300776	eval-mlogloss:1.344342
[6]	train-mlogloss:1.224587	eval-mlogloss:1.274713
[7]	train-mlogloss:1.155025	eval-mlogloss:1.210505
[8]	train-mlogloss:1.094254	eval-mlogloss:1.154902
[9]	train-mlogloss:1.038241	eval-mlogloss:1.103368
[10]	train-mlogloss:0.988977	eval-mlogloss:1.059299
[11]	train-mlogloss:0.943367	eval-mlogloss:1.018244
[12]	train-mlogloss:0.902526	eval-mlogloss:0.981581
[13]	train-mlogloss:0.863797	eval-mlogloss:0.946934
[14]	train-mlogloss:0.828207	eval-mlogloss:0.915240
[15]	train-mlogloss:0.795833	eval-mlog


Training with params : 
{'n_estimators': 500, 'min_child_weight': 8, 'subsample': 0.9, 'eta': 0.1, 'num_class': 9, 'nthread': 4, 'colsample_bytree': 0.8, 'max_depth': 13, 'silent': 1, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}
	Score 0.4621866637300336




[266]	train-mlogloss:0.131695	eval-mlogloss:0.462187
Stopping. Best iteration:
[256]	train-mlogloss:0.136276	eval-mlogloss:0.461729



{'subsample': 0.80000000000000004}